# Static Simulation for the Reputation Score

Reference: https://hackmd.io/8CSLDfKNSAyyl-OfEzOdDQ

In [33]:
import scipy.stats as st
from dataclasses import dataclass
import seaborn as sns
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt

In [25]:
Filecoin = float
Days = float


@dataclass
class Deal():
    payment: Filecoin
    duration: Days
    collateral: Filecoin
    finished: bool
    slashed: bool

    def deal_payoff(self,
                    slash_penalty: float) -> float:
        S = slash_penalty if self.slashed else 1.0
        return (self.duration * self.collateral * self.finished * S / self.payment)


@dataclass
class Provider():
    deals: list[Deal]

    def reputation_payoff(self,
                          slash_penalty: float,
                          alpha: float) -> float:
                          
        return sum(d.deal_payoff(slash_penalty) ** alpha
                   for d
                   in self.deals)


In [26]:
N_PROVIDERS = 1_000
DEALS_PER_PROVIDER_MU = 10

providers = []
for _ in range(N_PROVIDERS):
    N_provider_deals = st.poisson(mu=DEALS_PER_PROVIDER_MU).rvs()

    
    durations = st.gamma.rvs(a=2, scale=20, size=N_provider_deals)
    collaterals = st.gamma.rvs(a=1, scale=1, size=N_provider_deals)
    payments = collaterals / 10
    finished = st.bernoulli.rvs(p=0.5, size=N_provider_deals)
    slashed = st.bernoulli.rvs(p=0.01, size=N_provider_deals)

    deals_data = zip(payments, durations, collaterals, finished, slashed)

    deals = [Deal(*deal_data)
             for deal_data
             in deals_data]

    providers.append(Provider(deals))


In [28]:
provider_reputation_payoffs = [p.reputation_payoff(slash_penalty=-10, alpha=1/2)
                               for p 
                               in providers]



/var/folders/y4/dvc6qr5j30s3frwztjt5shg40000gn/T/ipykernel_28473/1179977817.py:27: RuntimeWarning:

invalid value encountered in double_scalars



In [29]:
px.histogram(provider_reputation_payoffs)

In [40]:
provider_ranking = np.argsort(provider_reputation_payoffs)
provider_norm_ranking = 100 * provider_ranking / max(provider_ranking)

px.scatter(x=provider_norm_ranking, 
            y=provider_reputation_payoffs)

In [31]:
provider_norm_ranking[0:10]

array([98.3983984 , 58.05805806, 60.06006006, 40.54054054, 24.62462462,
       27.52752753, 71.97197197, 33.23323323, 54.45445445, 81.08108108])

In [32]:
provider_reputation_payoffs[0:10]

[199.15041120122098,
 134.2047820394357,
 171.23115433672982,
 73.82758319344342,
 45.68947403423415,
 138.57136299093827,
 73.05471232787458,
 99.5351888176833,
 67.72830669035014,
 nan]